# [LintCode](https://www.lintcode.com/problem/) Problems

## 22. Flatten List (Easy)

**Description**

Given a list, each element in the list can be a list or integer. flatten it into a simply list with integers.

ⓘ If the element in the given list is a list, it can contain list too.

**Example**

Given [1, 2, [1, 2]], return [1, 2, 1, 2].

Given [4, [3, [2, [1]]]], return [4, 3, 2, 1].

**Challenge**

Do it in non-recursive.

<u>**Solution**</u>


+ **方法1**: non-recursive version

In [ ]:
def flatten(nestedList):
    flat = True
    nl = nestedList
    while flat:
        flat = False
        new_nl = []
        for i in nl:
            if type(i) == int:
                new_nl.append(i)
            else:
                for i_j in i:
                    new_nl.append(i_j)
                    flat = True
        nl = new_nl
    return nl

In [ ]:
nestedList = [1, 2, [1, 2]]
%time nl1 = flatten(nestedList)
print nl1
nestedList = [4, [3, [2, [1]]]]
%time nl2 = flatten(nestedList)
print nl2

+ **方法2**: recursive version

In [ ]:
def flatten(nestedList):
    res = []
    for i in nestedList:
        if type(i) == int:
            res.append(i)
        else:
            tmp = flatten(i)
            for j in tmp:
                res.append(j)
    return res

In [ ]:
nestedList = [1, 2, [1, 2]]
%time nl1 = flatten(nestedList)
print nl1
nestedList = [4, [3, [2, [1]]]]
%time nl2 = flatten(nestedList)
print nl2

## 24. LFU Cache (Hard)

**Description**

LFU (Least Frequently Used) is a famous cache eviction algorithm.

For a cache with capacity k, if the cache is full and need to evict a key in it, the key with the least frequently used will be kicked out.

Implement set and get method for LFU cache.

**Example**

Given capacity=3

```
set(2, 2)
set(1, 1)
get(2)
>> 2
get(1)
>> 1
get(2)
>> 2
set(3, 3)
set(4, 4)
get(3)
>> -1
get(2)
>> 2
get(1)
>> 1
get(4)
>> 4
```

<u>**Solution**</u>

+ LRU

首先提一下**LRU(Least Recently Used)**, 即最近最少使用. 那么这个缓存器主要有两个成员函数, get和set, 其中get函数是通过输入key来获得value, 如果成功获得后, 这对(key, value)升至缓存器中最常用的位置(顶部), 如果key不存在, 则返回-1. 而set函数是插入一对新的(key, value), 如果原缓存器中有该key, 则需要先删除掉原有的, 将新的插入到缓存器的顶部. 如果不存在, 则直接插入到顶部. 若加入新的值后缓存器超过了容量, 则需要删掉一个最不常用的值, 也就是底部的值. 具体实现时我们需要三个私有变量, cap, l和m, 其中cap是缓存器的容量大小, l是保存缓存器内容的列表, m是哈希表, 保存关键值key和缓存器各项的迭代器之间映射, 方便我们以O(1)的时间内找到目标项. 

+ LFU

这道题让我们实现最近不常用页面置换算法**LFU(Least Frequently Used)**, 而前面的**LRU Cache**, 让我们求最近最少使用页面置换算法LRU(Least Recnetly Used). 两种算法虽然名字看起来很相似, 但是其实是不同的. 顾名思义, LRU算法是首先淘汰最长时间未被使用的页面, 而LFU是先淘汰一定时间内被访问次数最少的页面. 举例说明, 比如说我们的cache大小为3, 然后我们按顺序存入5, 4, 5, 4, 5, 7, 这时候cache刚好被装满了, 因为put进去之前存在的数不会占用额外地方. 那么此时我们想再put进去一个8, 如果使用LRU算法, 应该将4删除, 因为4最久未被使用, 而如果使用LFU算法, 则应该删除7, 因为7被使用的次数最少, 只使用了一次.

这道题比前面LRU要麻烦一些, 因为LRU只要用个list把数字按时间顺序存入, 链表底部的位置总是最久未被使用的, 每次删除底部的值即可. 而这道题不一样, 由于需要删除最少次数的数字, 那么我们必须要统计每一个key出现的次数, 所以我们用一个哈希表m来记录当前数据{key, value}和其出现次数之间的映射, 这样还不够, 为了方便操作, 我们需要把相同频率的key都放到一个list中, 那么需要另一个哈希表freq来建立频率和一个里面所有key都是当前频率的list之间的映射. 由于题目中要我们在O(1)的时间内完成操作了, 为了快速的定位freq中key的位置, 我们再用一个哈希表iter来建立key和freq中key的位置之间的映射. 最后当然我们还需要两个变量cap和minFreq, 分别来保存cache的大小, 和当前最小的频率.

In [37]:
class KeyNode(object):
    def __init__(self, key, value, freq = 1):
        self.key = key
        self.value = value
        self.freq = freq
        self.prev = self.next = None

class FreqNode(object):
    def __init__(self, freq, prev, next):
        self.freq = freq
        self.prev = prev
        self.next = next
        self.first = self.last = None

class LFUCache(object):

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity
        self.keyDict = dict()
        self.freqDict = dict()
        self.head = None

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key in self.keyDict:
            keyNode = self.keyDict[key]
            value = keyNode.value
            self.increase(key, value)
            return value
        return -1

    def set(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        if self.capacity == 0:
            return
        if key in self.keyDict:
            self.increase(key, value)
            return
        if len(self.keyDict) == self.capacity:
            self.removeKeyNode(self.head.last)
        self.insertKeyNode(key, value)

    def increase(self, key, value):
        """
        Increments the freq of an existing KeyNode<key, value> by 1.
        :type key: str
        :rtype: void
        """
        keyNode = self.keyDict[key]
        keyNode.value = value
        freqNode = self.freqDict[keyNode.freq]
        nextFreqNode = freqNode.next
        keyNode.freq += 1
        if nextFreqNode is None or nextFreqNode.freq > keyNode.freq:
            nextFreqNode = self.insertFreqNodeAfter(keyNode.freq, freqNode)
        self.unlinkKey(keyNode, freqNode)
        self.linkKey(keyNode, nextFreqNode)

    def insertKeyNode(self, key, value):
        """
        Inserts a new KeyNode<key, value> with freq 1.
        :type key: str
        :rtype: void
        """
        keyNode = self.keyDict[key] = KeyNode(key, value)
        freqNode = self.freqDict.get(1)
        if freqNode is None:
            freqNode = self.freqDict[1] = FreqNode(1, None, self.head)
            if self.head:
                self.head.prev = freqNode
            self.head = freqNode
        self.linkKey(keyNode, freqNode)

    def delFreqNode(self, freqNode):
        """
        Delete freqNode.
        :rtype: void
        """
        prev, next = freqNode.prev, freqNode.next
        if prev: prev.next = next
        if next: next.prev = prev
        if self.head == freqNode: self.head = next
        del self.freqDict[freqNode.freq]

    def insertFreqNodeAfter(self, freq, node):
        """
        Insert a new FreqNode(freq) after node.
        :rtype: FreqNode
        """
        newNode = FreqNode(freq, node, node.next)
        self.freqDict[freq] = newNode
        if node.next: node.next.prev = newNode
        node.next = newNode
        return newNode

    def removeKeyNode(self, keyNode):
        """
        Remove keyNode
        :rtype: void
        """
        self.unlinkKey(keyNode, self.freqDict[keyNode.freq])
        del self.keyDict[keyNode.key]

    def unlinkKey(self, keyNode, freqNode):
        """
        Unlink keyNode from freqNode
        :rtype: void
        """
        next, prev = keyNode.next, keyNode.prev
        if prev: prev.next = next
        if next: next.prev = prev
        if freqNode.first == keyNode: freqNode.first = next
        if freqNode.last == keyNode: freqNode.last = prev
        if freqNode.first is None: self.delFreqNode(freqNode)

    def linkKey(self, keyNode, freqNode):
        """
        Link keyNode to freqNode
        :rtype: void
        """
        firstKeyNode = freqNode.first
        keyNode.prev = None
        keyNode.next = firstKeyNode
        if firstKeyNode: firstKeyNode.prev = keyNode
        freqNode.first = keyNode
        if freqNode.last is None: freqNode.last = keyNode

In [45]:
capacity = 3
obj = LFUCache(capacity)
obj.set(2, 2)
obj.set(1, 1)
print obj.get(2)
print obj.get(1)
print obj.get(2)
obj.set(3, 3)
obj.set(4, 4)
print obj.get(3)
print obj.get(2)
print obj.get(1)
print obj.get(4)

2
1
2
-1
2
1
4


## 28. Search a 2D Matrix (Easy)

**Description**

Write an efficient algorithm that searches for a value in an m x n matrix.

This matrix has the following properties:

+ Integers in each row are sorted from left to right.
+ The first integer of each row is greater than the last integer of the previous row.

**Example**

Consider the following matrix:

```
[
    [1, 3, 5, 7],
    [10, 11, 16, 20],
    [23, 30, 34, 50]
]
```

Given target = 3, return true.

**Challenge**

O(log(n) + log(m)) time

<u>**Solution**</u>


+ **方法1**: 两次二分查找

对第0列使用二分查找(O(log(m))), 找到元素可能在的行. 再在该行使用二分查找(O(log(n))).

In [ ]:
import numpy as np

In [ ]:
def searchMatrix(matrix, target):
    m = len(matrix)
    n = len(matrix[0])
    if m == 0 or n == 0:
        return False
    low = 0
    high = m - 1
    while low <= high:
        mid = low + (high - low) / 2
        if matrix[mid, 0] == target:
            return True
        elif matrix[mid, 0] < target:
            low = mid + 1
        else:
            high = mid - 1
    if high < 0:
        return False
    row = high
    low = 0
    high = n - 1
    while low <= high:
        mid = low + (high - low) / 2
        if matrix[row, mid] == target:
            return True
        elif matrix[row, mid] < target:
            low = mid + 1
        else:
            high = mid - 1
    return False

In [ ]:
matrix1 = np.array([[1, 3, 5, 7], [10, 11, 16, 20], [23, 30, 34, 50]], dtype=np.integer)
target = 3
%time searchMatrix(matrix1, target)

+ **方法2**: 一次二分查找

如果我们按S型遍历该二维数组, 可以得到一个有序的一维数组, 那么我们只需要用一次二分查找法, 而关键就在于坐标的转换, 如何把二维坐标和一维坐标转换是关键点:

把一个长度为L的一维数组转化为$m \times n$的二维数组($m \times n = L$)后,

1. 原二维数组中下标为[i, j]的元素将出现在一维数组中下标为$i \times (m + 1) + j$的位置;
2. 原一维数组中下标为k的元素将出现在二维数组中下标为[k / n, k % n]的位置.

In [ ]:
def searchMatrix(matrix, target):
    m = len(matrix)
    n = len(matrix[0])
    if m == 0 or n == 0:
        return False
    if target < matrix[0, 0] or target > matrix[m - 1, n - 1]:
        return False
    left = 0
    right = m * n - 1
    while left <= right:
        mid = left + (right - left) / 2
        if matrix[mid / n, mid % n] == target:
            return True
        elif matrix[mid / n, mid % n] < target:
            left = mid + 1
        else:
            right = mid - 1
    return False

In [ ]:
matrix1 = np.array([[1, 3, 5, 7], [10, 11, 16, 20], [23, 30, 34, 50]], dtype=np.integer)
target = 3
%time searchMatrix(matrix1, target)

## 29. Interleaving String (Medium)

**Description**

Given three strings: s1, s2, s3, determine whether s3 is formed by the interleaving of s1 and s2.

**Example**

For s1 = "aabcc", s2 = "dbbca"

When s3 = "aadbbcbcac", return true.

When s3 = "aadbbbaccc", return false.

**Challenge**

O($n^2$) time or better

<u>**Solution**</u>

[Dynamical Programming](https://en.wikipedia.org/wiki/Dynamic_programming)

+ **方法1**:

首先, 这道题的大前提是字符串s1和s2的长度和必须等于s3的长度, 如果不等于, 肯定返回false. 那么当s1和s2是空串的时候, s3必然是空串, 则返回true. 所以直接给dp[0][0]赋值true, 然后若s1和s2其中的一个为空串的话, 那么另一个肯定和s3的长度相等, 则按位比较, 若相同且上一个位置为True, 赋True, 其余情况都赋False, 这样的二维数组dp的边缘就初始化好了. 下面只需要找出递推公式来更新整个数组即可, 我们发现, 在任意非边缘位置dp[i][j]时, 它的左边或上边有可能为True或是False, 两边都可以更新过来, 只要有一条路通着, 那么这个点就可以为True. 那么我们得分别来看, 如果左边的为True, 那么我们去除当前对应的s2中的字符串s2[j - 1] 和 s3中对应的位置的字符相比(计算对应位置时还要考虑已匹配的s1中的字符), 为s3[j - 1 + i], 如果相等, 则赋True, 反之赋False. 而上边为True的情况也类似, 所以可以求出递推公式为：

$$dp[i][j] = (dp[i - 1][j] \&\& s1[i - 1] == s3[i - 1 + j]) || (dp[i][j - 1] \&\& s2[j - 1] == s3[j - 1 + i])$$

其中dp[i][j] 表示的是 s2 的前 i 个字符和 s1 的前 j 个字符是否匹配 s3 的前 i+j 个字符.

In [ ]:
def isInterleave(s1, s2, s3):
    n1 = len(s1)
    n2 = len(s2)
    if n1 + n2 != len(s3):
        return False
    dp = [[False for i in range(n1 + 1)] for j in range(n2 + 1)]
    dp[0][0] = True
    for i in range(1, n1 + 1):
        dp[i][0] = dp[i - 1][0] and (s1[i - 1] == s3[i - 1])
    for i in range(1, n2 + 1):
        dp[0][i] = dp[0][i - 1] and (s2[i - 1] == s3[i - 1])
    for i in range(1, n1 + 1):
        for j in range(1, n2 + 1):
            dp[i][j] = (dp[i - 1][j] and s1[i - 1] == s3[i - 1 + j]) or (dp[i][j - 1] and s2[j - 1] == s3[j - 1 + i])
    return dp[n1][n2]

In [ ]:
s1 = 'aabcc'
s2 = 'dbbca'
s3 = 'aadbbcbcac'
%time dp = isInterleave(s1, s2, s3)
print dp
s3 = 'aadbbbaccc'
%time dp = isInterleave(s1, s2, s3)
print dp

+ **方法2**: 简单改进

我们也可以把for循环合并到一起, 用if条件来处理边界情况, 整体思路和上面的方法没有太大的区别.

In [ ]:
def isInterleave(s1, s2, s3):
    n1 = len(s1)
    n2 = len(s2)
    if n1 + n2 != len(s3):
        return False
    dp = [[False for i in range(n1 + 1)] for j in range(n2 + 1)]
    for i in range(n1 + 1):
        for j in range(n2 + 1):
            if i == 0 and j == 0:
                dp[i][j] = True
            elif i == 0:
                dp[i][j] = dp[i][j - 1] and (s2[j - 1] == s3[i + j - 1])
            elif j == 0:
                dp[i][j] = dp[i - 1][j] and (s1[i - 1] == s3[i + j - 1])
            else:
                dp[i][j] = (dp[i - 1][j] and s1[i - 1] == s3[i + j - 1]) or (dp[i][j - 1] and s2[j - 1] == s3[i + j -1])
    return dp[n1][n2]

In [ ]:
s1 = 'aabcc'
s2 = 'dbbca'
s3 = 'aadbbcbcac'
%time dp = isInterleave(s1, s2, s3)
print dp
s3 = 'aadbbbaccc'
%time dp = isInterleave(s1, s2, s3)
print dp

+ **方法3**: 优化的DFS

我们使用一个哈希集合, 用来保存匹配失败的情况, 我们分别用变量i, j和k来记录字符串s1, s2和s3匹配到的位置, 初始化的时候都传入0. 在递归函数中, 首先根据i和j, 算出key值, 由于我们的哈希集合中只能放一个数字, 而我们要encode两个数字i和j, 所以通过用i乘以s3的长度再加上j来得到key, 此时我们看, 如果key已经在集合中, 直接返回false, 因为集合中存的是无法匹配的情况. 然后先来处理corner case的情况, 如果i等于s1的长度了, 说明s1的字符都匹配完了, 此时s2剩下的字符和s3剩下的字符可以直接进行匹配了, 所以我们直接返回两者是否能匹配的bool值. 同理, 如果j等于s2的长度了, 说明s2的字符都匹配完了, 此时s1剩下的字符和s3剩下的字符可以直接进行匹配了, 所以我们直接返回两者是否能匹配的bool值. 如果s1和s2都有剩余字符, 那么当s1的当前字符等于s3的当前字符, 那么调用递归函数, 注意i和k都加上1, 如果递归函数返回true, 则当前函数也返回true; 还有一种情况是, 当s2的当前字符等于s3的当前字符, 那么调用递归函数, 注意j和k都加上1, 如果递归函数返回true, 那么当前函数也返回true. 如果匹配失败了, 则将key加入集合中, 并返回false即可.

> 这里补充介绍集合:

set是一个**无序**且**不重复**的元素集合.

集合对象是一组无序排列的可哈希的值, 集合成员可以做字典中的键. 集合支持用in和not in操作符检查成员, 由len()内建函数得到集合的基数(大小), 用for循环迭代集合的成员. 但是因为集合本身是无序的, 不可以为集合创建索引或执行切片(slice)操作, 也没有键(keys)可用来获取集合中元素的值.

set和dict一样, 只是没有value, 相当于dict的key集合, 由于dict的key是不重复的且key是不可变对象, 因此set也有如下特性:

1. 不重复;
2. 元素为不可变对象.

+ 集合创建

In [ ]:
s = set()
print s
s = {11,22,33,44}                   #注意在创建空集合的时候只能使用s=set()，因为s={}创建的是空字典
print s
a = set('boy')
b = set(['y', 'b', 'o', 'o'])
c = set({"k1":'v1', 'k2':'v2'})
d = {'k1', 'k2', 'k2'}
e = {('k1', 'k2', 'k2')}
print(a, type(a))
print(b, type(b))
print(c, type(c))
print(d, type(d))
print(e, type(e))

+ 集合比较

In [ ]:
se = {11, 22, 33}
be = {22, 55}
tmp1 = se.difference(be)           # 找到se中存在, be中不存在的集合, 返回新值
print(tmp1)
print(se)

tmp2 = se.difference_update(be)    # 找到se中存在, be中不存在的集合, 覆盖掉se
print(tmp2)
print(se)

+ 集合删除

In [ ]:
se = {11, 22, 33}
se.discard(11)
se.discard(44)                     # 移除不存在的元素不会报错
print(se)

se = {11, 22, 33}
se.remove(11)
#se.remove(44)                     # 移除不存在的元素会报错
print(se)

se = {11, 22, 33}                  # 移除末尾元素并把移除的元素赋给新值
tmp = se.pop()
print(tmp)
print(se)

+ 交集

In [ ]:
se = {11, 22, 33}
be = {22, 55}

tmp1 = se.intersection(be)         # 取交集，赋给新值
print(tmp1)
print(se)

tmp2 = se.intersection_update(be)  # 取交集并更新自己
print(tmp2)
print(se)

+ 集合判断

In [ ]:
se = {11, 22, 33}
be = {22}

print(se.isdisjoint(be))           # 判断是否不存在交集(有交集False, 无交集True)
print(se.issubset(be))             # 判断se是否是be的子集合
print(se.issuperset(be))           # 判断se是否是be的父集合

+ 集合合并

In [ ]:
se = {11, 22, 33}
be = {22}

tmp1 = se.symmetric_difference(be) # 合并不同项, 并赋新值
print(tmp1)
print(se)

tmp2 = se.symmetric_difference_update(be)  # 合并不同项, 并更新自己
print(tmp2)
print(se)

+ 并集

In [ ]:
se = {11, 22, 33}
be = {22, 44, 55}

tmp = se.union(be)                # 取并集, 并赋新值
print(se)
print(tmp)

+ 集合更新

In [ ]:
se = {11, 22, 33}
be = {22, 44, 55}

se.update(be)                    # 把se和be合并, 得出的值覆盖se
print(se)
se.update([66, 77])              # 可增加迭代项
print(se)

+ 集合转换

In [ ]:
se = set(range(4))
li = list(se)
tu = tuple(se)
st = str(se)
print(se, type(se))
print(li, type(li))
print(tu, type(tu))
print(st, type(st))

In [ ]:
def helper(s1, i, s2, j, s3, k, s):
    key = i * len(s3) + j
    if key in s:
        return False
    if i == len(s1):
        return s2[j:] == s3[k:]
    if j == len(s2):
        return s1[i:] == s3[k:]
    if (s1[i] == s3[k] and helper(s1, i + 1, s2, j, s3, k + 1, s)) or (s2[j] == s3[k] and helper(s1, i, s2, j + 1, s3, k + 1, s)):
        return True
    s.update({key})
    return False 
    
def isInterleave(s1, s2, s3):
    n1 = len(s1)
    n2 = len(s2)
    if n1 + n2 != len(s3):
        return False
    s = set()
    return helper(s1, 0, s2, 0, s3, 0, s)

In [ ]:
s1 = 'aabcc'
s2 = 'dbbca'
s3 = 'aadbbcbcac'
%time dp = isInterleave(s1, s2, s3)
print dp
s3 = 'aadbbbaccc'
%time dp = isInterleave(s1, s2, s3)
print dp

+ **方法4**: BFS

这里我们需要用队列queue来辅助运算, 如果将**方法1**中的二维数组dp列出来的True/False图当作一个迷宫的话, 那么BFS的目的就是要从(0, 0)位置找一条都是True的路径通到(n1, n2)位置, 这里我们还要使用哈希集合, 不过此时保存到是已经遍历过的位置, 队列中还是存key值, key值的encode方法跟上面DFS解法的相同, 初始时放个0进去. 然后我们进行while循环, 循环条件除了q不为空, 还有一个是k小于n3, 因为匹配完s3中所有的字符就结束了. 然后由于是一层层的遍历, 所以要直接循环queue中元素个数的次数, 在for循环中, 对队首元素进行解码, 得到i和j值, 如果i小于n1, 说明s1还有剩余字符, 如果s1当前字符等于s3当前字符, 那么把s1的下一个位置i+1跟j一起加码算出key值, 如果该key值不在于集合中, 则加入集合, 同时加入队列queue中; 同理, 如果j小于n2, 说明s2还有剩余字符, 如果s2当前字符等于s3当前字符, 那么把s2的下一个位置j+1跟i一起编码算出key值, 如果该key值不在于集合中, 则加入集合, 同时加入队列queue中. for循环结束后, k自增1. 最后如果匹配成功的话, 那么queue中应该只有一个(n1, n2)的key值, 且k此时等于n3, 所以当queue为空或者k不等于n3的时候都要返回false.

In [5]:
from Queue import Queue

In [15]:
def isInterleave(s1, s2, s3):
    n1 = len(s1)
    n2 = len(s2)
    n3 = len(s3)
    if n1 + n2 != n3:
        return False
    k = 0
    s = set()
    q = Queue()
    q.put(0)
    while q.empty() != True and k < n3:
        len_q = q.qsize()
        for t in range(len_q):
            front = q.get()
            i = front / n3
            j = front % n3
            if i < n1 and s1[i] == s3[k]:
                key = (i + 1) * n3 + j
                if key not in s:
                    s.update({key})
                    q.put(key)
            if j < n2 and s2[j] == s3[k]:
                key = i * n3 + j + 1
                if key not in s:
                    s.update({key})
                    q.put(key)
        k += 1
    return q.empty() != True and k == n3

In [16]:
s1 = 'aabcc'
s2 = 'dbbca'
s3 = 'aadbbcbcac'
%time dp = isInterleave(s1, s2, s3)
print dp
s3 = 'aadbbbaccc'
%time dp = isInterleave(s1, s2, s3)
print dp

CPU times: user 872 µs, sys: 543 µs, total: 1.41 ms
Wall time: 850 µs
True
CPU times: user 650 µs, sys: 617 µs, total: 1.27 ms
Wall time: 922 µs
False


## 30. Insert Interval (Easy)

**Description**

Given a non-overlapping interval list which is sorted by start point.

Insert a new interval into it, make sure the list is still in order and non-overlapping (merge intervals if necessary).

**Example**

Insert (2, 5) into [(1, 2), (5, 9)], we get [(1, 9)].

Insert (3, 4) into [(1, 2), (5, 9)], we get [(1, 2), (3, 4), (5, 9)].

<u>**Solution**</u>

用pos记录newInterval应该插入的位置. 顺序遍历intervals中的元素, 若当前interval的end比newInterval的start还小, 则将当前interval加入res, 同时pos＋1; 若比newInterval大, 则直接加入res; 若有overlap, 则需要merge, newInterval的start取两者间小的, end取两者间大的. 最后在pos的位置插入newInterval即可. 

In [ ]:
class Interval(object):
    def __init__(self, start, end):
        self.start = start
        self.end = end

def insert(intervals, newInterval):
    if newInterval == None or intervals == None:
        return intervals
    res = []
    pos = 0
    for interval in intervals:
        if interval.end < newInterval.start:
            res.append(interval)
            pos += 1
        elif interval.start > newInterval.end:
            res.append(interval)
        else:
            newInterval.start = min(interval.start, newInterval.start)
            newInterval.end = max(interval.end, newInterval.end)
    res = res[:pos] + [newInterval] + res[pos :]
    return res

In [ ]:
intervals = [Interval(1, 2), Interval(5, 9)]
newInterval = Interval(2, 5)
%time res = insert(intervals, newInterval)
for interval in res:
    print "(%d, %d)" % (interval.start, interval.end)
newInterval = Interval(3, 4)
%time res = insert(intervals, newInterval)
for interval in res:
    print "(%d, %d)" % (interval.start, interval.end)